#### GAT模型transductive learning(Cora数据集)

### Layers.py
建立单层的GAT网络
* 定义weight matrix对原有特征表示进行线性变化
* 需要实现self-attention机制
* 需要加权求和得到新的特征表示

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
!pip install tensorboardX

     |████████████████████████████████| 204kB 2.8MB/s 


**repeat的用法还是不理解**

In [0]:
x = torch.tensor([1, 2, 3])
# 参数：sizes (torch.Size or python:int...) 
#– The number of times to repeat this tensor along each dimension
print(x.shape)          # torch.Size([3])
print(x.repeat(1,4))
print(x.repeat(1,4).shape)    # torch.Size([1, 12])
print(x.repeat(4, 2))
print(x.repeat(4,2).shape)    # torch.Size([4, 6])
print(x.repeat(4, 2, 1).shape) # torch.Size([4, 2, 3])?????
print(x.repeat(4, 2, 1))    # 猜测最左边是第一个维度

print(x.repeat(1, 2, 4).shape) # torch.Size([1, 2, 12])?????
print(x.repeat(1, 2, 4))    # 猜测最左边是第一个维度

torch.Size([3])
tensor([[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]])
torch.Size([1, 12])
tensor([[1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3]])
torch.Size([4, 6])
torch.Size([4, 2, 3])
tensor([[[1, 2, 3],
         [1, 2, 3]],

        [[1, 2, 3],
         [1, 2, 3]],

        [[1, 2, 3],
         [1, 2, 3]],

        [[1, 2, 3],
         [1, 2, 3]]])
torch.Size([1, 2, 12])
tensor([[[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3],
         [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]]])


**如何将目标节点与邻居结点的特征拼接在一起？**
***
* step1:将所有可能的组合都拼接在一起

* Graph Feature: $\mathbf{H}=\left\{\vec{h}_{1}, \vec{h}_{2}, \ldots, \vec{h}_{N}\right\}^{T}$ （node_num,in_dim）

* H.permute(1,N): (node_num,in_dim*node_num)

$\left\{\vec{h}_{1}, \vec{h}_{1}, \ldots, \vec{h}_{1}\right\}$

$\left\{\vec{h}_{2}, \vec{h}_{2}, \ldots, \vec{h}_{2}\right\}$

——————省略号

$\left\{\vec{h}_{N}, \vec{h}_{N}, \ldots, \vec{h}_{N}\right\}$

**permute之后的矩阵每行将原结点特征重复N次**

* H.permute(1,N).view(N*N,-1)，得到一个列向量，维度为($nodeNum^{2}$,in_dim)

* a_input=torch.cat([H.repeat(1, N).view(N * N, -1),H.repeat(N, 1)],dim=1)

上面就是得到所有结点向量表示的组合，（结点1，结点2）与（结点2，结点1）算是2种不同的组合。

最终拼接的维度是($N^{2}$, in_dim+in_dim),每行是2个结点拼接的特征表示。

***


In [9]:
torch.manual_seed(5) 
H = torch.LongTensor([[1,2,4,5],[4,3,2,9],[1,1,1,1]]) 
N = H.size()[0]
print(N)
print(H.shape)    # H: (node_num,out_dim)
print(H.repeat(1,N))
print(H.repeat(1,N).shape)
print(H.repeat(N,1))
print(H.repeat(N,1).shape)
a_input=torch.cat([H.repeat(1, N).view(N * N, -1),H.repeat(N, 1)],dim=1)
print(a_input)

3
torch.Size([3, 4])
tensor([[1, 2, 4, 5, 1, 2, 4, 5, 1, 2, 4, 5],
        [4, 3, 2, 9, 4, 3, 2, 9, 4, 3, 2, 9],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
torch.Size([3, 12])
tensor([[1, 2, 4, 5],
        [4, 3, 2, 9],
        [1, 1, 1, 1],
        [1, 2, 4, 5],
        [4, 3, 2, 9],
        [1, 1, 1, 1],
        [1, 2, 4, 5],
        [4, 3, 2, 9],
        [1, 1, 1, 1]])
torch.Size([9, 4])
tensor([[1, 2, 4, 5, 1, 2, 4, 5],
        [1, 2, 4, 5, 4, 3, 2, 9],
        [1, 2, 4, 5, 1, 1, 1, 1],
        [4, 3, 2, 9, 1, 2, 4, 5],
        [4, 3, 2, 9, 4, 3, 2, 9],
        [4, 3, 2, 9, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 2, 4, 5],
        [1, 1, 1, 1, 4, 3, 2, 9],
        [1, 1, 1, 1, 1, 1, 1, 1]])


**激活函数**
***
torch.nn.ELU(alpha=1.0, inplace=False)

**注意：**这个函数是element-wise operation,即元素中**逐个激活**

$\operatorname{ELU}(x)=\max (0, x)+\min (0, \alpha *(\exp (x)-1))$
***

torch.nn.LeakyReLU(negative_slope=0.01, inplace=False)

该函数也是element_wise operation:

$\text { LeakyReLU }(x)=\max (0, x)+\text { negative_slope } * \min (0, x)$
***





In [0]:
class GraphAttentionLayer(nn.Module):
  def __init__(self,in_dim,out_dim,dropout,alpha,concat=True):
    super(GraphAttentionLayer,self).__init__()
    # network structure parameters
    # 这里使用self保存参数是为了方便forward函数使用
    self.in_dim = in_dim
    self.out_dim = out_dim
    self.dropout = dropout
    self.alpha = alpha
    self.concat = concat

    # network module
    # --linear transformation: weight matrix (in_dim,out_dim)
    self.W = nn.Parameter(torch.zeros(size=(in_dim,out_dim)))
    # -- feed-forward neural network: attention weight vector
    self.a = nn.Parameter(torch.zeros(size=(2*out_dim,1)))
    # activate function
    self.leakyrelu = nn.LeakyReLU(negative_slope=self.alpha)

    self.init_params() # initialzie parameters



  def init_params(self):
    nn.init.xavier_uniform_(self.W.data,gain=1.414) 
    # 这里的gain是如何计算出来的
    nn.init.xavier_uniform_(self.a.data,gain=1.414)

  # input: node feature representation
  # adj: adjacency matrix
  def forward(self,input,adj):
    # input：(node_num,in_dim)
    # adj: (node_num,node_num)
    # H: (node_num,out_dim)
    # linear transformation
    H = torch.mm(input,self.W)
    # N: number of graph nodes
    N = H.size()[0]

    # attention_input: all (target_node_representation) || (other node representation)
    # two step is attention mechansim stage1: similiarity(score) calculation
    attention_input=torch.cat([H.repeat(1, N).view(N * N, -1),H.repeat(N, 1)],dim=1).view(N,-1,2*self.out_dim)
    attention_coefficient = self.leakyrelu(torch.matmul(attention_input,self.a).squeeze(2))
    # attention_coefficient:[N,N],注意这里建立的权重要与邻接矩阵对应起来

    # two step is to mask attention,just pay attention to neighbourhood node.
    zero_matrix = -9e15*torch.ones_like(attention_coefficient)
    # define zero vector, not true zero,but very small number
    masked_coefficient = torch.where(adj>0,attention_coefficient,zero_matrix)
    # masked_coefficient：[N,N]

    # calculate attention weight
    a_weight = F.softmax(masked_coefficient,dim=1)
    a_weight = F.dropout(a_weight,self.dropout,training=self.training)
    # self.training的来源是哪里？
    # torch.nn.functional.dropout(input, p=0.5, training=True, inplace=False)
    H_prime = torch.matmul(a_weight,H) 
    # H_prime:(node_num,out_dim)

    if self.concat: # conact next layer
      return F.elu(H_prime)
    else:
      return H_prime
  # information of layer
  def __repr__(self):
    return self.__class__.__name__ + ' (' + str(self.in_dim) + ' -> ' + str(self.out_dim) + ')'

In [11]:
torch.manual_seed(5)
test_feature = torch.randn((10,100))
adj = torch.LongTensor([1,0,1,0,1,0,1,1,0,0]).repeat(10,1)
print(adj.shape)

layer_test = GraphAttentionLayer(in_dim=100,out_dim=10,dropout=0.5,alpha=0.01,concat=True)
print(layer_test.__repr__()) 
test_output = layer_test(test_feature,adj)
print(test_output.shape)

torch.Size([10, 10])
GraphAttentionLayer (100 -> 10)
torch.Size([10, 10])


### model.py

**API使用**
***
add_module(name, module)：添加子模块，可以使用self.name访问模块
***
普通的softmax(input, dim=None, _stacklevel=3, dtype=None)
* This function doesn’t work directly with NLLLoss, which expects the Log to be computed between the Softmax and itself. Use log_softmax instead (it’s faster and has better numerical properties).

log_softmax：这个函数更适用于NLLLoss

**上面2个函数使用要注意**
***

**模型介绍：**
* 模型分为2层，第一层是图网络层，第二层用于分类，先使用一个单独的注意力头去计算出C features(C是结点的种类数)，接着使用一个softmax激活。
* 论文中提出给每层的输入引入0.6的dropout，解释是**在每个训练迭代中，每个节点都暴露在随机采样的邻域中**（at each training iteration, each node is exposed to a stochastically sampled neighborhood）
* 论文中还提到在训练中不太准确的引入$L2$正则项，这一点该notebook中没有采纳，实验效果没有变差。

In [0]:
class GAT(nn.Module):
  def __init__(self,in_dim,hidden_dim,n_class,dropout,alpha,n_heads):
    """Dense version of GAT"""
    super(GAT,self).__init__()
    self.dropout = dropout
    self.multi_attentions=[GraphAttentionLayer(in_dim,hidden_dim,dropout,alpha,concat=True) for _ in range(n_heads)]
    # 将定义的多个图卷积层作为子模块添加进去
    for i,layer in enumerate(self.multi_attentions):
      self.add_module('attention_layer_{}'.format(i),self.multi_attentions[i])
    # 最后网络的输出层
    # 将multi-head attention的各个输出连接在一起作为输入,有什么意义？
    # 这里之所以这样做是论文中EXPERIMENTAL SETUP中这样设置的
    # 使用单独的注意力头计算，输出维度与结点种类数相同
    self.out_layer=GraphAttentionLayer(hidden_dim*n_heads,n_class,dropout,alpha,False)


  def forward(self,x,adj):
    x = F.dropout(x,self.dropout,training=self.training)
    # concatenate features [n_node,n_head*out_dim]
    x = torch.cat([layer(x,adj) for layer in self.multi_attentions],dim=1)
    x = F.dropout(x,self.dropout,training=self.training)
    # 这里直接将拼接后的目标结点特征作为输入单个注意力头的输入
    # 作者认为这是一种average操作
    x = F.elu(self.out_layer(x,adj))
    return F.log_softmax(x,dim=1)  # 输出结点概率分布

**测试GAT模型代码**



In [13]:
myGAT=GAT(in_dim=100,hidden_dim=10,n_class=3,dropout=0.5,alpha=0.1,n_heads=2)
torch.manual_seed(5)
test_feature = torch.randn((10,100))
adj = torch.LongTensor([1,0,1,0,1,0,1,1,0,0]).repeat(10,1)
print(adj.shape)
test_output = myGAT(test_feature,adj)
print(test_output.shape)

torch.Size([10, 10])
torch.Size([10, 3])


### 数据处理

In [0]:
import scipy.sparse as sp
import random
from tqdm import tqdm
import time
import os
import glob   # lists of files matching given patterns, just like the Unix shell
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter

In [0]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

这里的mx是$\tilde{A}=A+I$,即邻接矩阵+单位矩阵

rowsum:$\tilde{A}$的度矩阵$\tilde{D}$

r_mat_inv_sqrt:$\tilde{D}^{-\frac{1}{2}}$

最终结果：$(\tilde{A}\tilde{D}^{-\frac{1}{2}})^{T}\tilde{D}^{-\frac{1}{2}}$

实际论文中表达：$\tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}$

这是对预处理步骤，原论文称为**renormalization trick**，为了alleviate数值不稳定以及梯度爆炸和消失问题

见论文：SEMI-SUPERVISED CLASSIFICATION WITH GRAPH CONVOLUTIONAL NETWORKS

此处代码疑问：为什么与原论文有差异？？？


In [0]:
def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)
    # 这里的transpose()是转置操作

公式表达：$\tilde{D}^{-1}\tilde{A}$

In [0]:
# GCN-pytorch代码的实现，这里作为对比不使用
def normalize(mx):
    """Row-normalize sparse matrix"""
    # 这里之所以是行，是因为每一行都是一个特征
    rowsum = np.array(mx.sum(1))  # 得到所有元素的和
    # print(rowsum.shape)   # (2708, 1)
    r_inv = np.power(rowsum, -1).flatten()  # 取倒数,0的倒数为inf,flatten降维成列表
    r_inv[np.isinf(r_inv)] = 0.    # 将inf替换为0
    r_mat_inv = sp.diags(r_inv)    # 使用r_inv创立对角矩阵
    mx = r_mat_inv.dot(mx)         # 将对角矩阵乘以原矩阵进行特征化
    return mx

**这里搞不懂为什么要normalize features???**

In [0]:
def normalize_features(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [0]:
def load_data(path="./data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize_features(features)
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    adj = torch.FloatTensor(np.array(adj.todense()))
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
    print("finishing load datasets!")

    return adj, features, labels, idx_train, idx_val, idx_test

In [20]:
path='/content/drive/My Drive/GNN/data/cora/'
adj, features, labels, idx_train, idx_val, idx_test=load_data(path=path)


Loading cora dataset...
finishing load datasets!


In [0]:
print(adj.shape)
print(features.shape)
print(labels.shape)
print(idx_train.shape)  # 140,包含0-139的值
print(idx_train)  
print(idx_val.shape)   # 300
print(idx_val)      # 包含200-499各种值
print(idx_test.shape)  # 1000
# 包含500-1499范围的各种值

### train.py



In [0]:
seed=72
epochs=10000
lr=0.005
weight_decay=5e-4
hidden=8
nb_heads=8
dropout=0.6
alpha=0.2
patience=100

In [0]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

model=GAT(in_dim=features.shape[1],
    hidden_dim=hidden,
    n_class=int(labels.max())+1,
    dropout=dropout,
    alpha=alpha,
    n_heads=nb_heads)

optimizer = optim.Adam(model.parameters(),lr=lr,weight_decay=weight_decay) # 模型参数优化器

In [25]:
print(labels[idx_train])

tensor([0, 1, 6, 6, 2, 2, 5, 0, 0, 5, 0, 4, 2, 3, 0, 0, 6, 0, 0, 0, 5, 6, 4, 2,
        6, 5, 0, 2, 0, 1, 3, 0, 3, 0, 6, 6, 0, 6, 4, 6, 5, 2, 4, 0, 3, 6, 0, 3,
        2, 3, 3, 2, 5, 4, 5, 4, 5, 6, 0, 0, 6, 3, 2, 6, 3, 4, 3, 2, 0, 3, 5, 2,
        0, 6, 2, 3, 2, 0, 5, 5, 0, 2, 6, 0, 2, 6, 1, 1, 5, 5, 0, 0, 6, 2, 2, 2,
        3, 0, 5, 2, 4, 0, 0, 6, 0, 2, 1, 2, 2, 5, 0, 0, 0, 0, 5, 2, 3, 2, 6, 1,
        4, 0, 0, 0, 0, 0, 2, 0, 6, 4, 5, 2, 6, 0, 0, 1, 1, 2, 2, 4])


In [0]:
# 这里的cuda()是将所有张量转换到GPU计算
model.cuda()
features = features.cuda()
adj = adj.cuda()
labels = labels.cuda()
idx_train = idx_train.cuda()
idx_val = idx_val.cuda()
idx_test = idx_test.cuda()

In [0]:
# features, adj, labels = Variable(features), Variable(adj), Variable(labels) # 这一步不需要
# 老版本pytorch代码需要用到Varialbe，新版本基本不需要

<class 'torch.Tensor'>
False


In [0]:
log_dir = '/content/sample_data/log'
writer = SummaryWriter(log_dir=log_dir + '/' + time.strftime('%H:%M:%S', time.gmtime()))

In [0]:
# 这里features,adj作为全局变量加载
# 注意此处的反向传播方法

def train(epoch):
    t = time.time()
    model.train()   
    optimizer.zero_grad()
    output = model(features, adj)
    print(output.shape)    # torch.Size([2708, 7])
    # train model: calculate loss value
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])

    loss_train.backward() 
    optimizer.step()   

    # evaluate model
    model.eval()
    output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])

    # print and recored information
    train_loss = loss_train.detach().item()
    dev_loss = loss_val.detach().item()
    train_acc = acc_train.detach().item()
    dev_acc = acc_val.detach().item()
    time_cost = time.time() - t

    # print('Epoch: {:04d}'.format(epoch+1),
    #       'loss_train: {:.4f}'.format(train_loss),
    #       'acc_train: {:.4f}'.format(train_acc),
    #       'loss_val: {:.4f}'.format(dev_loss),
    #       'acc_val: {:.4f}'.format(dev_acc),
    #       'time: {:.4f}s'.format(time_cost))
    c = epoch + 1
    writer.add_scalar("loss/train", train_loss,c)
    writer.add_scalar("loss/dev", dev_loss,c)
    writer.add_scalar("acc/train", train_acc,c)
    writer.add_scalar("acc/dev", dev_acc,c)
    writer.add_scalar("epoch_time",time_cost,c)

    return dev_loss   # determine the model effect

In [33]:
# output:结点的概率分布矩阵
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

# output = torch.tensor([[0.4,0.6],[0.7,0.3],[0.1,0.9]])
# labels = torch.LongTensor([0,1,1])
# preds = output.max(1)[1].type_as(labels)
# test = output.max(1)

def GPU_info(empty=False):
  print("memory allocated is {}".format(torch.cuda.memory_allocated()))
  print("cached memory is {}".format(torch.cuda.memory_cached()))
  if empty:  
    print("releases all unused cached memory from PyTorch")
    torch.cuda.empty_cache()
GPU_info(empty=True)

memory allocated is 5076528640
cached memory is 6782189568
releases all unused cached memory from PyTorch


In [0]:

dev_loss_list = []
model_dir = '/content/sample_data/model/'
bad_counter = 0 
best_epoch = 1 
best_dev_loss = float('inf') 
t_total = time.time()
for epoch in tqdm(range(epochs)):
  dev_loss = train(epoch)
  dev_loss_list.append(dev_loss)
  if dev_loss<best_dev_loss:
    best_dev_loss = dev_loss
    torch.save(model.state_dict(),model_dir+'{}.pkl'.format(epoch))
    best_epoch = epoch
  else:
    bad_counter += 1
  if bad_counter == 100:
    break
  # remove save model file before best epoch
  files = glob.glob(model_dir+'*.pkl')
  for file in files:
    tmp = file.split('/')[-1]
    tmp = tmp.split('.')[0]
    epoch_nb = int(tmp)
    if epoch_nb < best_epoch:
      os.remove(file)
  
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print("best epoch is {}".format(best_epoch))

In [43]:
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load(model_dir+'{}.pkl'.format(best_epoch)))

Loading 433th epoch


<All keys matched successfully>

In [42]:
model.eval()
output = model(features, adj)
loss_test = F.nll_loss(output[idx_test], labels[idx_test])
acc_test = accuracy(output[idx_test], labels[idx_test])
test_loss = loss_test.detach().item()
test_acc = acc_test.detach().item()
print("Test set results:","loss= {:.4f}".format(test_loss),"accuracy= {:.4f}".format(test_acc)) 
# 准确率是0.8410，原论文是83.0+(-)0.7

Test set results: loss= 0.6890 accuracy= 0.8410


### 总结
关于模型
* 模型是GAT模型，出自论文GRAPH ATTENTION NETWORKS-ICLR-2018
***
关于本实验
* 本实验数据集是cora数据集，目的是用于评估网络的transductive learning

* cora数据集共有2708个结点，由于是transuductive learning，所有结点
的特征都会作为网络的输入，但该notebook实际训练时，使用140个结点用于参数优化，使用300个结点作为dev,使用1000个结点作为test,**这里原论文中是使用500个作为dev，并且训练时数据每一类结点都为20个，7类共140个，其余的基本一致**
* 模型共2层，第一层时图网络层，注意头为8个，每个注意力维度为8，所以最终经过第一层网络输出的结点维度为64维，
